<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=300 height=300 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2023<br>
<font color=3C99D size=5>
    Practical Assignment 2 Solution - CSP <br>
<font color=696880 size=4>
    Mohammad Mahdi Abootorabi 

____

# Q1- CSP (50 Points)

In this question, we are going to plan for some future events in the country. There are some halls that can host different kinds of events, such as concerts, sports events, etc.

Two halls are considered adjacent if the distance between them is less than a threshold. Two adjacent halls cannot host the same event.

With these constraints, complete the code below and assign an event to each hall to satisfy every constraint.



## Example
Look at this example carefully:<br>
<img src="Images/CSP_example.png" width="500"/>
  - There are $6$ halls and $3$ kinds of events.
  - The halls in which each event can be held (for example, sport events can be held only at hall 1, 4 and 6).
  - Adjacent halls (we show adjacent halls here with a line between them) cannot host the same event (for example, halls 1 and 2 both cannot host concert events).

## Note
  - You _must_ implement and use AC-3 algorithm.
  - You will _probably_ need to utilize the heuristic algorithms you have learned (e.g. MRV and LCV) or consider nearly tree-structured graphs approach in order to pass all tests in an appropriate time.

## Input
  - The first line consists of $N$ (count of halls) and $M$ (count of event kinds).
  - In the following $M$ lines, the $i$ th line ($1 \le i \le M$) is a list of preferred halls for the $i$ th event (separated by space).
  - In the next line, $E$, the total number of adjacent constraints (edges in the previous graph) are given.
  - in each of the following $E$ lines, a pair of hall numbers (separated by space) is given.
$$1 \le N \le 50$$
$$1 \le M \le 50$$

### Sample Input
This sample describes the previous example (sport=1, concert=2, fashion=3).
```
6 3
1 4 6
1 2 3 5 6
3 4 5
5
1 2
2 3
3 4
3 5
3 6
```

## Output
In the only line, print:
  - list of one appropriate assignment ordered by hall number (separated by space).
  - `NO` if there isn't.

### Sample Output
One possible solution to the example above is {Hall1=sport, Hall2=concert , Hall3=fashion, Hall4=sport, Hall5=concert, Hall6=concert}. So:
```
1 2 3 1 2 2
```
In another example, If Hall3 was not an option for fashion, then the answer would be:
```
NO
```
Because then hall 2 or 3 would be empty.

## Your code
**Note:** It's OK to change the signature for the given functions and the given structure is just a suggestion to help you with the implementation. (you can't remove or add any cells)

In [18]:
# Auxiliary functions

In [19]:
def ac_3(csp_domains, csp_constraints):
    # Initialize the queue with all the arcs in the CSP
    queue = [(xi, xj) for xi in csp_domains for xj in csp_domains if xi != xj]

    # Keep iterating until the queue is empty
    while queue:
        (xi, xj) = queue.pop(0)
        # If the domain of xi is revised, we need to check its neighbors
        if revise(csp_domains, xi, xj):
            # If the domain of xi is empty, there is no solution
            if not csp_domains[xi]:
                return False
            # Add neighboring arcs back to the queue
            for xk in csp_domains:
                if xk != xi and (xk, xi) in csp_constraints:
                    queue.append((xk, xi))
    return True

def revise(csp_domains, xi, xj):
    # Indicates whether we revised the domain of xi
    revised = False
    # Check every value in the domain of xi
    for x in csp_domains[xi]:
        # If no value in the domain of xj satisfies the constraint with x, remove x
        if all(not is_different(x, y) for y in csp_domains[xj]):
            csp_domains[xi].remove(x)
            revised = True
    return revised

def is_different(x, y):
    # Check if two values are different
    return x != y

In [20]:
def backtrack(csp_domains, csp_constraints, assignment):
    # If assignment is complete then return the assignment
    if len(assignment) == len(csp_domains):
        return assignment

    # Select an unassigned hall
    unassigned = [hall for hall in csp_domains if hall not in assignment]
    hall = select_unassigned_variable(unassigned, csp_domains)

    # Consider events for hall in order
    for event in order_domain_values(hall, csp_domains, assignment, csp_constraints):
        # If value is consistent with assignment
        if is_consistent(hall, event, assignment, csp_constraints):
            # Assign it and continue to build up the assignment
            assignment[hall] = event
            result = backtrack(csp_domains, csp_constraints, assignment)
            # If we did not fail, return the result
            if result:
                return result
            # If we failed, remove the assignment and backtrack
            del assignment[hall]

    # If no assignment could be made, return failure
    return False

def backtracking_search(csp_domains, csp_constraints):
    # Start backtracking with an empty assignment
    assignment = {}
    return backtrack(csp_domains, csp_constraints, assignment)

# Helper functions needed by the backtrack function

def select_unassigned_variable(unassigned, csp_domains):
    # Selects the next variable to assign using the Minimum Remaining Values (MRV) heuristic
    return min(unassigned, key=lambda hall: len(csp_domains[hall]))

def order_domain_values(hall, csp_domains, assignment, csp_constraints):
    # Orders the values in the domain of the variable using the Least Constraining Value (LCV) heuristic
    return sorted(csp_domains[hall], key=lambda event: lcv_score(hall, event, csp_domains, assignment, csp_constraints))

def is_consistent(hall, event, assignment, csp_constraints):
    # Check if an assignment is consistent with the assignments made so far
    for (hall1, hall2) in csp_constraints:
        if (hall == hall1 and hall2 in assignment and assignment[hall2] == event) or \
           (hall == hall2 and hall1 in assignment and assignment[hall1] == event):
            return False
    return True

def lcv_score(hall, event, csp_domains, assignment, csp_constraints):
    # Calculate the number of choices left for neighboring variables if we choose a particular value for the current variable
    count = 0
    for (hall1, hall2) in csp_constraints:
        if hall == hall1 or hall == hall2:
            neighbor = hall2 if hall == hall1 else hall1
            if event in csp_domains[neighbor]:
                count += 1
    return count

In [22]:
import Helper_codes.csp_helper as csp
import time
import networkx as nx
import matplotlib.pyplot as plt


def plot_test_case(result, n, next_e_lines, test_num):   # Do not change this function. This is for plotting the assignment. 
    if result == 'NO':
        return
    reult_list = result.split()
    reult_list = list(map(int, reult_list))
    if len(reult_list) >= 15:
        print(f'Too many colors to plot for test {test_num + 1}')
        return

    G = nx.Graph()
    G.add_nodes_from(range(1, n + 1))
    for edge in next_e_lines:
        G.add_edge(edge[0], edge[1])
    colors = ['red', 'green', 'blue', 'yellow', 'orange', 'purple', 'pink', 'brown', 'gray', 'black', 'cyan', 'magenta', 'olive', 'teal']
    
    color_map = []
    for i in range(1, n + 1):
        color_map.append(colors[reult_list[i - 1] - 1])
          
    nx.draw(G, node_color=color_map, with_labels=True)
    plt.show()
 

TIME_LIMIT = 3

tests = csp.get_all_tests(prefix='csp_')
tests_passed = 0
for test_num, test in enumerate(tests):
    n, m, m_next_lines, e, next_e_lines = csp.scan_test_input(test)
    #################################################################
    # (Point: 5% of total score)                                    #
    # under this comment section implement a code to handle inputs  #
    #################################################################
    csp_domains = {hal: set(range(1, m + 1)) for hal in range(1, n + 1)}
    for event, halls in enumerate(m_next_lines, start=1):
        for hal in halls:
            csp_domains[hal] = csp_domains[hal].intersection(set(m_next_lines[event - 1]))
    csp_constraints = set()
    for edge in next_e_lines:
        csp_constraints.add((edge[0], edge[1]))
        csp_constraints.add((edge[1], edge[0]))

    start_time = time.time()
    result = backtracking_search(csp_domains, csp_constraints)
    print(f'test {test_num + 1} is completed')
    print(f'assignment: {result}')
    total_time = time.time() - start_time
    if csp.is_result_valid(test, result) and total_time < TIME_LIMIT:
        tests_passed += 1
    else:
        print(f'test {test} failed. time elapsed= {total_time}')
    plot_test_case(result, n, next_e_lines, test_num)
    print('----------------------------------------------------------')
# (Point: 50% of your total score)                                    #
print(f'Score = {tests_passed / len(tests) * 100}%')

test 1 is completed
assignment: False


TypeError: argument of type 'bool' is not iterable